### Fit a Random Forest CART Regression Model 
#### 1. EDA
#### 2. Fit a random forest model on the business train data

In [2]:
import numpy as np
import random
import time
import imp
import json
import sklearn as sk
import random
from random import seed
from random import random
from random import randrange
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import json
import csv
import unicodecsv
import codecs
from collections import defaultdict
import sys
import csv
#imp.reload(sys)
#sys.stdout = codecs.getwriter('utf-8')(sys.stdout)
#sys.stderr = codecs.getwriter('utf-8')(sys.stderr)
import pickle
from collections import Counter
import itertools
import nltk
import re
from nltk.probability import *
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import matplotlib
import matplotlib.pyplot as plt

# Cleaning the Business Data

In [54]:
df = pd.read_csv('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_business_train.csv')
pf = pd.read_csv('/Users/chin/Yelp Reviews/data/yelp_academic_dataset_business_test.csv')


In [56]:
df.columns

Index(['Unnamed: 0', 'neighborhood', 'business_id', 'hours', 'is_open',
       'address', 'attributes', 'categories', 'city', 'review_count', 'name',
       'longitude', 'state', 'stars', 'latitude', 'postal_code', 'type'],
      dtype='object')

In [4]:
df['attributes'].isnull()
choose = pf['attributes'].notnull()
hi = pf.copy()
hi = hi[choose]

We want to drop the rows that have no data in the "attributes" column, because this means the business has almost no data to use.

In [7]:
df = df.dropna(subset=['attributes'])
pf = pf.dropna(subset=['attributes'])

In [8]:
# This is if we want to give specific values to words (in particular things like 0.5 etc.)

def fixdata(df):
    words = {
    "no" : 0, "False":0, "free":1, "full_bar": 1, "casual": 0, "True":1, "none": 0, "beer_and_wine" : 1,"yes_free":1, 
    "average":1,"loud":.75,"quiet":1,"very_loud":0,"yes":1,"outdoor":1,"paid":0.5, "dressy":1,'1':1,"2":2,"3":3,"4":4,
    "yes_corkage":1, "formal":1}
    for i in df.index:
        dicl = dictionize(df['attributes'][[i]][i], "_")
        for j in dicl.keys():
            if j in df.columns:
                if type(dicl[j]) == int:
                    k = dicl[j]
                else:
                    k = words[dicl[j]]
                    
                df.set_value(i,j,k)
            else:
                df[j] = np.zeros(df.shape[0])
    return df


In [9]:
# This puts the attribute values directly into the a new column for each attribute

def fixdata2(df):
    words = {
    "no" : 0, "False":0, "free":1, "full_bar": 1, "casual": 0, "True":1, "none": 0, "beer_and_wine" : 1,"yes_free":1, 
    "average":1,"loud":.75,"quiet":1,"very_loud":0,"yes":1,"outdoor":1,"paid":0.5, "dressy":1,'1':1,"2":2,"3":3,"4":4,
    "yes_corkage":1, "formal":1}
    for i in df.index:
        dicl = dictionize(df['attributes'][[i]][i], "_")
        for j in dicl.keys():
            k = dicl[j]
            if j in df.columns:
                df.ix[i,j]=k
            else:
                df[j] = list("0" for i in range(0,df.shape[0]))
                df.ix[i,j]=k
    return df

In [10]:
# This turns the attribute column into a dictionary. A helper function.
def dictionize(column,delim):
    val = {}
    lst = column[1:-1].split(",")
    contains2 = 0
    for j in range(0,len(lst)):
        lst2 = lst[j].split(": ")
        first_contains = 0
        for k in range(0, len(lst2)):
            if '{' in lst2[k]:
                first_contains = 1
                contains2 = 1
                keep = stripignore(lst2[0])
            elif '}' in lst2[k]:
                contains2 = 0
            else:
                k = 9
        if first_contains == 1:
            val[keep + delim + stripignore(lst2[1])] = stripignore(lst2[2])
        elif contains2 == 1:
            val[keep + delim + stripignore(lst2[0])] = stripignore(lst2[1])
        else:
            if '}' in lst2[1]:
                val[keep + delim + stripignore(lst2[0])] = stripignore(lst2[1]) 
            else:
                val[stripignore(lst2[0])] = stripignore(lst2[1])
    return val

                

In [11]:
# Strips a string of any random things like { } or ] 
def stripignore(string):
    new_string =""
    ignore = ['"',' ', '{',"'", '}','[',']']
    for i in range(0,len(string)):
        if string[i] in ignore:
            k = 9
        else:
            new_string = new_string + string[i]
    return(new_string)
        

In [12]:
new_df = fixdata2(df)
new_pf = fixdata2(pf)


In [61]:
new_df['type'][[11]][11]

'business'

In [14]:
dicl = dictionize(df['attributes'][[5]][5], "_")
dicl

{'Alcohol': 'full_bar',
 'Ambience_casual': 'True',
 'Ambience_classy': 'False',
 'Ambience_divey': 'False',
 'Ambience_hipster': 'False',
 'Ambience_intimate': 'False',
 'Ambience_romantic': 'False',
 'Ambience_touristy': 'False',
 'Ambience_trendy': 'False',
 'Ambience_upscale': 'False',
 'BikeParking': 'True',
 'BusinessAcceptsCreditCards': 'True',
 'BusinessParking_garage': 'False',
 'BusinessParking_lot': 'False',
 'BusinessParking_street': 'True',
 'BusinessParking_valet': 'False',
 'BusinessParking_validated': 'False',
 'Caters': 'True',
 'GoodForKids': 'True',
 'GoodForMeal_breakfast': 'False',
 'GoodForMeal_brunch': 'False',
 'GoodForMeal_dessert': 'False',
 'GoodForMeal_dinner': 'True',
 'GoodForMeal_latenight': 'True',
 'GoodForMeal_lunch': 'True',
 'HasTV': 'True',
 'NoiseLevel': 'average',
 'OutdoorSeating': 'False',
 'RestaurantsAttire': 'casual',
 'RestaurantsDelivery': 'True',
 'RestaurantsGoodForGroups': 'True',
 'RestaurantsPriceRange2': '2',
 'RestaurantsReservations

In [15]:
new_df.columns

Index(['Unnamed: 0', 'neighborhood', 'business_id', 'hours', 'is_open',
       'address', 'attributes', 'categories', 'city', 'review_count', 'name',
       'longitude', 'state', 'stars', 'latitude', 'postal_code', 'type',
       'Alcohol', 'Ambience_romantic', 'Ambience_intimate', 'Ambience_classy',
       'Ambience_hipster', 'Ambience_divey', 'Ambience_touristy',
       'Ambience_trendy', 'Ambience_upscale', 'Ambience_casual', 'BYOB',
       'BYOBCorkage', 'BikeParking', 'BusinessAcceptsCreditCards',
       'BusinessParking_garage', 'BusinessParking_street',
       'BusinessParking_validated', 'BusinessParking_lot',
       'BusinessParking_valet', 'Caters', 'GoodForKids', 'GoodForMeal_dessert',
       'GoodForMeal_latenight', 'GoodForMeal_lunch', 'GoodForMeal_dinner',
       'GoodForMeal_breakfast', 'GoodForMeal_brunch', 'HasTV', 'NoiseLevel',
       'OutdoorSeating', 'RestaurantsAttire', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'Rest

We can see that we now have many more columns, one for each value in the attribute column.

In [16]:
new_df.shape

(2477, 69)

In [17]:
new_df.describe()

,Unnamed: 0,is_open,review_count,longitude,stars,latitude,postal_code
count,2477.000000,2477.000000,2477.000000,2477.000000,2477.000000,2477.000000,2477.000000
mean,72570.237384,0.728704,46.878482,-103.169226,3.383730,36.215702,68478.465079
std,41248.117285,0.444718,103.801780,14.932225,0.750609,2.974058,27744.049782
min,85.000000,0.000000,3.000000,-115.350808,1.000000,33.303505,15120.000000
25%,37641.000000,0.000000,8.000000,-115.089580,3.000000,33.560621,44120.000000
50%,73248.000000,1.000000,20.000000,-112.029639,3.500000,36.021088,85036.000000
75%,107597.000000,1.000000,50.000000,-81.702447,4.000000,36.218024,89101.000000
max,143978.000000,1.000000,3258.000000,-79.704755,5.000000,43.147071,89183.000000


In [52]:
new_df.to_csv('/Users/chin/Yelp Reviews/data/summarytables.csv')

In [53]:
summary = new_df.groupby('BusinessAcceptsCreditCards').aggregate(np.mean)
summary

,Unnamed: 0,is_open,review_count,longitude,stars,latitude,postal_code
BusinessAcceptsCreditCards,,,,,,,
0,74497.616279,0.430233,5.244186,-101.506810,3.127907,36.740808,65045.651163
False,77519.807692,0.750000,20.250000,-96.601160,3.711538,36.868889,53861.192308
True,72389.334331,0.739205,49.001283,-103.376369,3.385849,36.181873,68929.649423


Now, we want to see how many of the columns have missing data: this corresponds to either the restaurant not having enough data to have an attribute or that attribute not being important enough.

In [19]:
# creates a data frame that contains the % values that are actually reported
def percent_reporting(frame,start):
    store = {}
    for i in frame.columns[17:len(frame.columns)]:       
        summary = frame.groupby(i).aggregate(len)
        store[i] = 1 - summary['hours'][0]/len(frame)
    return(pd.Series(store))

In [20]:
# This function drops the columns that don't meet a threshold % of reported values
def drop_columns(frame, threshold, begin):
    hi = percent_reporting(frame,begin)
    choose = pd.Series({i:1 for i in frame.columns[0:begin]})
    choose = choose.append(hi)
    choose2 = choose > threshold
    for i in frame.columns:
        if i not in choose2[choose2]:
            frame.drop(i, axis=1, inplace=True)
    return(frame)

We choose 58% as our threshold value.

In [21]:
new_df2 = drop_columns(new_df,.58,17)
new_pf2 = drop_columns(new_pf,.58,17)

We want only the columns that have at least 60% of the data reporting: Therefore, we will drop many of the columns.

new_df2.drop('Unnamed: 0', axis=1, inplace=True)
new_df2.drop('neighborhood', axis=1, inplace=True)
new_df2.drop('hours', axis=1, inplace=True)
new_df2.drop('address', axis=1, inplace=True)
new_df2.drop('city', axis=1, inplace=True)
new_df2.drop('name', axis=1, inplace=True)
new_df2.drop('state', axis=1, inplace=True)
new_df2.drop('longitude', axis=1, inplace=True)
new_df2.drop('latitude', axis=1, inplace=True)

In [22]:
# Not used
def drop_columns2(frame,drop_list):
    k = frame.copy()
    for i in frame.columns:
        if i in drop_list:
           k  = k.drop(i, axis=1)
    return(k)

In [23]:
# Creates dummy variables between keyword1 and keyword2
def chin_dummies(frame, keyword1, keyword2):
    frame = pd.get_dummies(frame, columns = frame.columns[frame.columns.get_loc(keyword1):(frame.columns.get_loc(keyword2)+1)])
    return(frame)

We want to create dummy variables for the attributes we created above.

In [24]:
new_df3 = chin_dummies(new_df2, "Alcohol","WiFi")
new_pf3 = chin_dummies(new_pf2, "Alcohol","WiFi")

In [25]:
new_pf3

,Unnamed: 0,Unnamed: 0.1,neighborhood,business_id,hours,is_open,address,attributes,categories,city,...,RestaurantsTableService_0,RestaurantsTableService_False,RestaurantsTableService_True,RestaurantsTakeOut_0,RestaurantsTakeOut_False,RestaurantsTakeOut_True,WiFi_0,WiFi_free,WiFi_no,WiFi_paid
0,0,300,Eastside,CCNp1qS8n2gh51X7Wadabw,NaN,0,3000 Paradise Rd,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","['Mexican', 'Restaurants']",Las Vegas,...,0,0,1,0,0,1,0,0,1,0
1,1,593,NaN,01aNlDhbMObjc9OdAHuNpQ,"['Monday 11:0-21:0', 'Tuesday 11:0-21:0', 'Wed...",1,14 W Pacific Ave,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","['Restaurants', 'Chinese']",Henderson,...,0,0,1,0,0,1,0,0,1,0
2,2,895,NaN,YalmY--dRI7hWdKutyyrlA,NaN,1,"5000 S Arizona Mills Cir, Ste 391","['BikeParking: True', 'BusinessAcceptsCreditCa...","['Food', 'Coffee & Tea']",Tempe,...,1,0,0,0,0,1,0,1,0,0
3,3,914,NaN,MV9fUwgI2AQa4uZArynRlA,NaN,1,"10707 Park Rd, Ste A","['Alcohol: full_bar', ""Ambience: {'romantic': ...","['Mexican', 'Restaurants']",Charlotte,...,0,0,1,0,0,1,0,0,1,0
4,4,1466,NaN,vu49Y9DNbjReqzFWkTI1xQ,"['Monday 8:0-16:30', 'Tuesday 8:0-16:30', 'Wed...",1,7585 E Gray Rd,"['BikeParking: True', 'BusinessAcceptsCreditCa...","['Food', 'Coffee & Tea']",Scottsdale,...,1,0,0,0,1,0,0,0,1,0
5,5,1892,Oakland,a3ZqjElMgAe924sbXZVcvA,"['Monday 11:0-19:0', 'Tuesday 11:0-19:0', 'Wed...",1,3404 5th Ave,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Restaurants', 'Pizza']",Pittsburgh,...,0,1,0,0,0,1,0,0,1,0
6,6,2992,NaN,oopfFjmzxgkDB0tt160nkg,"['Monday 11:0-22:0', 'Tuesday 11:0-22:0', 'Wed...",0,2501 Freeport Rd,"['Alcohol: full_bar', 'BusinessAcceptsCreditCa...","['Restaurants', 'Mexican']",Pittsburgh,...,0,0,1,1,0,0,1,0,0,0
7,7,3604,NaN,EwBOUo676Jtdc5AlEwp5CA,NaN,1,1402 East Chandler Blvd,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Restaurants', 'Mexican']",Phoenix,...,0,1,0,0,0,1,0,0,1,0
8,8,3861,NaN,8oVyhkGvcFd_pXwvxOzOxQ,"['Monday 10:0-22:0', 'Tuesday 10:0-22:0', 'Wed...",1,1726 W Bell Rd,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Pizza', 'Restaurants']",Phoenix,...,0,1,0,0,0,1,0,0,1,0
9,9,4113,Summerlin,jPvn-C8kNUcyGfjUtOc_oA,"['Monday 17:0-22:0', 'Tuesday 17:0-22:0', 'Wed...",0,10820 W Charleston Blvd,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","['Mexican', 'Restaurants']",Las Vegas,...,0,0,1,0,0,1,0,0,1,0


In [26]:
def fixdata3(frame):
    hold0 = []
    hold1 = []
    for i in frame.index:
        l = stripignore(frame['categories'][[i]][i].split(",")[0])
        p = stripignore(frame['categories'][[i]][i].split(",")[1])
        if l == "Restaurants" or l == "Food":
            hold0.append(l)
            hold1.append(p)
        else:
            hold0.append(p)
            hold1.append(l)
    frame["Type of Establishment"] = hold0
    frame["Category of Food"] = hold1
    return(frame)

In [27]:
new_df4 = fixdata3(new_df3)
new_pf4 = fixdata3(new_pf3)

In [28]:
new_df4 = chin_dummies(new_df4,"Type of Establishment","Category of Food")
new_pf4 = chin_dummies(new_pf4,"Type of Establishment","Category of Food")

In [50]:
new_df4

,Unnamed: 0,neighborhood,business_id,hours,is_open,address,attributes,categories,city,review_count,...,WiFi_0,WiFi_free,WiFi_no,WiFi_paid,Type of Establishment_Food,Type of Establishment_Restaurants,Category of Food_Chinese,Category of Food_Coffee&Tea,Category of Food_Mexican,Category of Food_Pizza
0,85,NaN,Ja4stXdNYr39u5CZHMNtjw,"['Monday 11:0-23:0', 'Tuesday 11:0-0:0', 'Wedn...",1,7228 E 1st Ave,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","['Mexican', 'Restaurants']",Scottsdale,480,...,0,1,0,0,0,1,0,0,1,0
1,114,Capitol,Gi48Z9ZPlEUgQU1xo_7Hjw,"['Monday 7:0-22:0', 'Tuesday 7:0-22:0', 'Wedne...",1,650 State St,"['BikeParking: True', 'BusinessAcceptsCreditCa...","['Coffee & Tea', 'Food']",Madison,39,...,0,1,0,0,1,0,0,1,0,0
2,117,University City,X02YNLa3Iofzwv5Le6QeYQ,NaN,0,8640 University City Blvd,"['Alcohol: none', 'BusinessAcceptsCreditCards:...","['Pizza', 'Restaurants']",Charlotte,5,...,1,0,0,0,0,1,0,0,0,1
3,152,NaN,4z2evNWOBm7b4-7Td0392A,"['Monday 5:0-22:0', 'Tuesday 5:0-22:0', 'Wedne...",1,"1817 E Guadalupe Rd, Ste 101","['BikeParking: True', 'BusinessAcceptsCreditCa...","['Food', 'Coffee & Tea']",Tempe,39,...,0,1,0,0,1,0,0,1,0,0
4,184,Centennial,kUntNQ5P9IrRzEoHdRxV-w,"['Monday 11:0-21:0', 'Tuesday 11:0-21:0', 'Wed...",0,7930 W Tropical Pkwy,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","['Restaurants', 'Pizza']",Las Vegas,74,...,0,0,1,0,0,1,0,0,0,1
5,225,The Hill District,ZYof0WYlilJ3pgWNSgmYKQ,"['Monday 10:0-2:0', 'Tuesday 10:0-2:0', 'Wedne...",1,1304 5th Ave,"['Alcohol: full_bar', ""Ambience: {'romantic': ...","['Pizza', 'Restaurants']",Pittsburgh,56,...,0,0,1,0,0,1,0,0,0,1
6,260,NaN,-vb_yx5QnIhpXUIdPVD2og,"['Monday 11:30-22:0', 'Tuesday 11:30-22:0', 'W...",0,"1930 N Decatur Blvd, Ste 1","['Alcohol: beer_and_wine', ""Ambience: {'romant...","['Restaurants', 'Chinese']",Las Vegas,7,...,0,0,1,0,0,1,1,0,0,0
7,269,University,3rkxTx8DoZSl7_FryhXCVQ,"['Monday 10:30-21:30', 'Tuesday 10:30-21:30', ...",0,4747 S Maryland Pkwy,"['Alcohol: none', ""Ambience: {'romantic': Fals...","['Restaurants', 'Chinese']",Las Vegas,34,...,0,0,1,0,0,1,1,0,0,0
9,585,Oakland,prZHC3I8JUI98n9iJVvDbw,"['Monday 5:30-21:0', 'Tuesday 5:30-21:0', 'Wed...",1,3618 Forbes Ave,"['BikeParking: True', 'BusinessAcceptsCreditCa...","['Coffee & Tea', 'Food']",Pittsburgh,19,...,0,1,0,0,1,0,0,1,0,0
10,658,Sunrise,tstimHoMcYbkSC4eBA1wEg,"['Monday 11:0-21:0', 'Tuesday 10:0-21:0', 'Wed...",1,6055 E Lake Mead Blvd,"['Alcohol: beer_and_wine', ""Ambience: {'romant...","['Restaurants', 'Mexican']",Las Vegas,120,...,0,0,1,0,0,1,0,0,1,0


In [51]:
new_df4.to_csv('/Users/chin/Yelp Reviews/data/summarytables.csv')

In [30]:
drop_list = ["Unnamed: 0",'Unnamed: 0.1',"business_id", "name","neighborhood","hours","is_open","address","attributes","categories","city","state","postal_code","type"]

new_df5 = drop_columns2(new_df4,drop_list)
new_pf5 = drop_columns2(new_pf4,drop_list)


In [31]:
def createtrainingtest(frame, number):
    frame['is_train'] = np.random.uniform(0, 1, len(frame)) <= number
    train, test = frame[frame['is_train']==True], frame[frame['is_train']==False]
    return train, test

In [32]:
new_pf5.columns

Index(['review_count', 'longitude', 'latitude', 'Alcohol_0',
       'Alcohol_beer_and_wine', 'Alcohol_full_bar', 'Alcohol_none',
       'Ambience_romantic_0', 'Ambience_romantic_False', 'Ambience_intimate_0',
       ...
       'WiFi_0', 'WiFi_free', 'WiFi_no', 'WiFi_paid',
       'Type of Establishment_Food', 'Type of Establishment_Restaurants',
       'Category of Food_Chinese', 'Category of Food_Coffee&Tea',
       'Category of Food_Mexican', 'Category of Food_Pizza'],
      dtype='object', length=118)

In [59]:
len(new_df5.columns)

118

In [34]:
train2, test2 = createtrainingtest(new_df5,.75)

In [35]:
train_y, test_y = train2['stars'], test2['stars']

In [36]:
train3 = train2.drop('stars', axis=1)
test3 = test2.drop('stars',axis = 1)
train = train3.drop('is_train',axis=1)
test = test3.drop('is_train',axis=1)

In [37]:
for i in new_pf5.columns:
    if i not in test.columns:
        new_pf5 = new_pf5.drop(i,axis = 1)

for i in test.columns:
    if i not in new_pf5.columns:
        new_pf5[i] = np.zeros(new_pf5.shape[0])

In [38]:
new_pf5.columns

Index(['review_count', 'longitude', 'latitude', 'Alcohol_0',
       'Alcohol_beer_and_wine', 'Alcohol_full_bar', 'Alcohol_none',
       'Ambience_romantic_0', 'Ambience_romantic_False', 'Ambience_intimate_0',
       ...
       'WiFi_paid', 'Type of Establishment_Food',
       'Type of Establishment_Restaurants', 'Category of Food_Chinese',
       'Category of Food_Coffee&Tea', 'Category of Food_Mexican',
       'Category of Food_Pizza', 'Ambience_romantic_True',
       'Ambience_upscale_True', 'BusinessParking_validated_True'],
      dtype='object', length=116)

In [1]:
train_y2 = train_y.apply(str)
train_y2

NameError: name 'train_y' is not defined

In [40]:
train_y

1       3.0
2       4.0
3       3.0
4       3.5
5       3.5
6       3.5
7       3.5
9       3.5
10      4.5
12      3.0
14      5.0
15      3.0
16      3.5
17      3.5
19      4.0
20      3.5
21      5.0
22      3.5
23      3.5
24      3.0
26      4.0
27      4.0
28      2.5
29      2.5
31      3.5
34      2.5
36      3.0
37      2.5
38      3.0
40      4.0
       ... 
2476    3.0
2477    5.0
2478    4.0
2479    4.0
2480    2.5
2482    4.5
2484    3.0
2485    4.0
2486    4.0
2487    3.5
2488    2.5
2489    3.5
2490    4.0
2491    2.5
2492    3.5
2493    5.0
2494    3.0
2495    2.5
2497    4.5
2498    3.0
2499    3.5
2500    3.0
2501    2.0
2503    3.0
2504    3.5
2505    5.0
2506    4.0
2507    3.0
2508    3.5
2509    4.5
Name: stars, dtype: float64

In [41]:
clr = RandomForestClassifier(n_jobs = 5, n_estimators = 1000)

In [42]:
clr.fit(train,train_y2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=5, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [43]:
clr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=5, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [44]:
predictions = clr.predict(test)

In [45]:
predictions

array(['2.5', '3.0', '3.5', '3.0', '3.5', '3.0', '4.0', '3.0', '3.0',
       '2.5', '3.0', '3.0', '3.0', '3.0', '3.0', '3.5', '3.0', '3.0',
       '2.5', '3.0', '2.5', '3.0', '3.0', '2.5', '3.0', '2.5', '3.0',
       '3.0', '3.0', '3.0', '3.0', '3.5', '3.0', '3.5', '3.0', '2.5',
       '2.5', '3.5', '3.0', '3.5', '3.0', '2.5', '4.0', '3.0', '2.5',
       '3.0', '2.5', '2.5', '4.0', '2.5', '3.0', '2.5', '3.0', '2.5',
       '3.5', '2.5', '3.0', '2.5', '3.0', '2.5', '3.0', '2.5', '2.5',
       '2.5', '3.0', '3.0', '3.0', '2.5', '2.5', '3.0', '4.0', '2.5',
       '3.0', '4.0', '3.0', '2.5', '2.5', '3.5', '3.0', '2.5', '2.5',
       '3.0', '3.0', '2.5', '4.0', '3.0', '3.0', '3.0', '3.0', '3.0',
       '3.0', '4.0', '2.5', '3.0', '3.5', '3.0', '4.0', '2.5', '3.5',
       '3.0', '3.0', '2.5', '4.0', '3.0', '3.0', '4.0', '3.0', '2.5',
       '3.0', '3.0', '4.0', '3.0', '3.0', '3.0', '4.0', '3.0', '2.5',
       '2.5', '4.0', '2.5', '2.5', '3.0', '2.5', '4.0', '3.0', '4.0',
       '3.0', '2.5',

In [46]:
predictions3 = predictions.astype(float)
predictions3

array([ 2.5,  3. ,  3.5,  3. ,  3.5,  3. ,  4. ,  3. ,  3. ,  2.5,  3. ,
        3. ,  3. ,  3. ,  3. ,  3.5,  3. ,  3. ,  2.5,  3. ,  2.5,  3. ,
        3. ,  2.5,  3. ,  2.5,  3. ,  3. ,  3. ,  3. ,  3. ,  3.5,  3. ,
        3.5,  3. ,  2.5,  2.5,  3.5,  3. ,  3.5,  3. ,  2.5,  4. ,  3. ,
        2.5,  3. ,  2.5,  2.5,  4. ,  2.5,  3. ,  2.5,  3. ,  2.5,  3.5,
        2.5,  3. ,  2.5,  3. ,  2.5,  3. ,  2.5,  2.5,  2.5,  3. ,  3. ,
        3. ,  2.5,  2.5,  3. ,  4. ,  2.5,  3. ,  4. ,  3. ,  2.5,  2.5,
        3.5,  3. ,  2.5,  2.5,  3. ,  3. ,  2.5,  4. ,  3. ,  3. ,  3. ,
        3. ,  3. ,  3. ,  4. ,  2.5,  3. ,  3.5,  3. ,  4. ,  2.5,  3.5,
        3. ,  3. ,  2.5,  4. ,  3. ,  3. ,  4. ,  3. ,  2.5,  3. ,  3. ,
        4. ,  3. ,  3. ,  3. ,  4. ,  3. ,  2.5,  2.5,  4. ,  2.5,  2.5,
        3. ,  2.5,  4. ,  3. ,  4. ,  3. ,  2.5,  4. ,  3. ,  3. ,  3. ,
        3. ,  2.5,  2.5,  3. ,  2.5,  4. ,  3. ,  3. ,  2.5,  3. ,  4. ,
        3. ,  3. ,  2.5,  3.5,  4. ,  4. ,  4. ,  3

In [47]:
mse = (predictions3 - test_y[choose]['stars'])**2

In [48]:
np.mean(mse)

0.8715277777777778